In [1]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/Users/phonghoang/Desktop/LLM Zoomcamp/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

es_client = Elasticsearch(
    "http://127.0.0.1:9200",
    headers={"Accept": "application/vnd.elasticsearch+json; compatible-with=8",
             "Content-Type": "application/vnd.elasticsearch+json; compatible-with=8"}
)

es_client.info()

ObjectApiResponse({'name': '57fa1f1ab41a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'WmS6faYtSQ6yXEirXIUWWQ', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 348.68it/s]


In [26]:
query = "How do copy a file to a Docker container?"

In [27]:
def elastic_search(query: str) -> list:
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])
        print(hit["_score"])
    return result_docs

In [31]:
results = elastic_search(query)

73.38676
66.688705
59.812744


In [32]:
context_template = """
Q: {question}
A: {text}
""".strip()

context_entries = []
for result in results:
    context_entry = context_template.format(
        question=result['question'],
        text=result['text']
    )
    context_entries.append(context_entry)
context = '\n\n'.join(context_entries)

In [33]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

question = "How do copy a file to a Docker container?"
prompt = prompt_template.format(question=question, context=context)

print(f"Length of the prompt: {len(prompt)}")

Length of the prompt: 1446


In [34]:
! pip install tiktoken

  Using cached tiktoken-0.9.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached regex-2024.11.6-cp310-cp310-macosx_11_0_arm64.whl.metadata (40 kB)
Using cached tiktoken-0.9.0-cp310-cp310-macosx_11_0_arm64.whl (1.0 MB)
Using cached regex-2024.11.6-cp310-cp310-macosx_11_0_arm64.whl (284 kB)

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [35]:
import tiktoken

# Get the encoding for gpt-4o
encoding = tiktoken.encoding_for_model("gpt-4o")

tokens = encoding.encode(prompt)

num_tokens = len(tokens)
print(f"Number of tokens: {num_tokens}")

Number of tokens: 320
